# Preparing to Train on DC2 Run 1.1p Data

**Owner:** Phil Marshall (@drphilmarshall)

**Last Run:** 2018-10-24 (by @drphilmarshall)

**Goals:** Use the DC2 Run 1.1p truth catalog, and DRP Object table, to construct a design matrix `X` (from the truth table) and corresponding response variables `y` (from the DRP Object table).

**Notes:** This notebook was made by adapting the following notebooks:
* Scott Daniel's DC2 Tutorial [`truth_gcr_intro.ipynb`](https://github.com/LSSTDESC/DC2-analysis/blob/master/tutorials/truth_gcr_intro.ipynb)
* ...

To run this notebook, follow the instructions to setup Jupyter-dev at NERSC: https://confluence.slac.stanford.edu/x/1_ubDQ

## Setup

In [ ]:
# import packages and methods that will be used in this notebook

import healpy
import numpy as np
import GCRCatalogs
from astropy.coordinates import SkyCoord
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

## Loading the Input Truth

In [ ]:
truth_catalog = GCRCatalogs.load_catalog('dc2_truth_run1.1_static')

You can list the quantities contained in the catalog using the same API as any other GCR-based catalog.

In [ ]:
all = truth_catalog.list_all_quantities(include_native=True)
all

The `get_quantity_info` method will give you access to descriptions of all of the native quantities in the catalog.  Note that the quantities `ugrizy` map directly to `mag_true_[ugrizy]'.

In [ ]:
for qty in all:
    info_dict = truth_catalog.get_quantity_info(qty)
    print(qty,info_dict)

We'll want to keep our training and test set small while developing, but then increase its size later. A good option for selecting a subset of objects is to filter by sky position, either relative to some reference point, or using the healpixellation.

In the cells below, we will query the catalog for all of the bright stars and galaxies in a 0.2 degree disc centered on `RA=54.6`, `Dec=-28.0`.

In the next cell, will define some methods (`filter_on_healpix` and `filter_on_dist`) needed to make that selection.  These methods will be passed into the catalog's `get_quantities` method with the `filters` kwarg.  The methods should accept numpy arrays and return a numpy array of booleans indicating whether or not each object in the input arrays passed the test.

We will use `healpy`'s `query_disc` method to find which healpixels overlap our region of interest.

In [ ]:
center_ra = 54.6
center_dec = -28.0
radius = 0.2

center_ra_rad = np.radians(center_ra)
center_dec_rad = np.radians(center_dec)

center_vec = np.array([np.cos(center_dec_rad)*np.cos(center_ra_rad),
                       np.cos(center_dec_rad)*np.sin(center_ra_rad),
                       np.sin(center_dec_rad)])

list_of_healpix = healpy.query_disc(2048, center_vec, np.radians(radius), nest=True, inclusive=True)

def filter_on_healpix(hp):
    return np.array([hh in list_of_healpix for hh in hp])

def angularSeparation(ra, dec, center_ra, center_dec):
    return SkyCoord(ra, dec, unit='deg').separation(SkyCoord(center_ra, center_dec, unit='deg')).deg

def filter_on_dist(ra, dec):
    return angularSeparation(ra, dec, center_ra, center_dec) < radius

Now we let's query the catalog for all objects in the region of interest with magnitude `r<27.0`.  This query functions like any other GRC-based catalog query with one exception.  The truth catalog is ultimately stored as a sqlite database.  This means that all `native_filters` (filters applied directly to the catalog's native quantities), should be phrased as strings which could be inserted into an SQL `WHERE` clause.  Because the `native_filters` are applied when the quantities loaded into memory (as opposed to `filters`, which are applied after the quantities have been loaded), we want these to be as restrictive as possible so as to limit the memory consumption of the loaded catalog.  The sqlite databases are indexed on `star`, `agn`, `sprinkled`, `healpix_2048` and `object_id`.

**Note:** We are aware that `nside==2048` healpixels may be too fine a resolution to helpfully limit the catalog query.  We are open to the idea of using a coarser resolution in future truth catalogs.

After the coarse spatial limits applied by the `native_filter` on `healpix_2048`, we use the `filter_on_healpix` and `filter_on_dist` methods to actually get sources in our region of interest.

In [ ]:
true_objects = truth_catalog.get_quantities(all,
                                            native_filters=['r<27.0',
                                                            'healpix_2048<=%d' % list_of_healpix.max(),
                                                            'healpix_2048>=%d' % list_of_healpix.min()],
                                            filters=[(filter_on_healpix, 'healpix_2048'),
                                                     (filter_on_dist, 'ra', 'dec')])

In [ ]:
print(len(true_objects['ra']))

In [ ]:
plt.scatter(true_objects['ra'], true_objects['dec'], alpha=0.01);

Now we will perform a similar spatial query for objects which were not added by the sprinkler (`sprinkled==0`) with magnitudes `r<27`.

***Note:** "the sprinkler" is a piece of code used to add an extra population of AGN and supernovae into the Utral Deep Drilling Field of DC2.  Objects added by the sprinkler will have no counterpart in the underlying protoDC2 extragalactic catalog, which is why we have added a `sprinkled` flag to the catalog.

In [ ]:
unsprinkled_true_objects = truth_catalog.get_quantities(all,
                                                           native_filters=['sprinkled==0',
                                                                           'r<27.0',
                                                                           'healpix_2048<=%d' % list_of_healpix.max(),
                                                                           'healpix_2048>=%d' % list_of_healpix.min()],
                                                           filters=[(filter_on_healpix, 'healpix_2048'),
                                                                    (filter_on_dist, 'ra', 'dec')])

In [ ]:
print(len(unsprinkled_true_objects['ra']))

## Finding the Matching LSST Objects

Now we need to load in a DRP Object table, and match its contents (spatially) to the input truth objects.

## Formatting into `X` and `y`

Now that we have a set of matched objects, we can reformat them into a design matrix `X` and the corresponding response variables `y`. We'll make these as simple `numpy` arrays.